In [ ]:
import os
from pyspark.sql.functions import col,regexp_replace, when, expr, 
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()
from pyspark import SparkFiles

#df=spark.read.csv(SparkFiles.get("combined_csv(1).csv"),sep=",",header=True,inferSchema=True)

url='https://project3group5.s3.amazonaws.com/SVDemo_combined.csv'

spark.sparkContext.addFile(url)
#df = spark.read.csv(SparkFiles.get)
df=spark.read.csv(SparkFiles.get("SVDemo_combined.csv"),sep=",")
df.show()

+-------+----+------------------+------+--------------------+-----+------------+------------+------------+
|    _c0| _c1|               _c2|   _c3|                 _c4|  _c5|         _c6|         _c7|         _c8|
+-------+----+------------------+------+--------------------+-----+------------+------------+------------+
|company|year|              race|gender|        job_category|count|valuation   |ownership   |eeo-1 status|
|23andMe|2016|Hispanic_or_Latino|  male|          Executives|    0|       $1.1b|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|            Managers|    1|       $1.1b|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|       Professionals|    7|       $1.1b|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|         Technicians|    0|       $1.1b|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|       Sales workers|    0|       $1.1b|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  ma

In [ ]:
df.columns

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8']

In [ ]:
# Create a new column called `Salary` where the values are the `Salary (1k)` * 1000
# Show bthe columns `Salary` and `Salary (1k)`
#df = df.withColumn("_c6", df["_c62)"] * 1000)
#df.select(["_c6", "_c62"]).show()

df_without = df.withColumn('_c6',
              when(df._c6.endswith('b'),regexp_replace(df._c6, 'b', '')))
df_nodollar = df_without.withColumn('_c6', regexp_replace('_c6','[$,]', '')).show()




+-------+----+------------------+------+--------------------+-----+----+------------+------------+
|    _c0| _c1|               _c2|   _c3|                 _c4|  _c5| _c6|         _c7|         _c8|
+-------+----+------------------+------+--------------------+-----+----+------------+------------+
|company|year|              race|gender|        job_category|count|null|ownership   |eeo-1 status|
|23andMe|2016|Hispanic_or_Latino|  male|          Executives|    0| 1.1|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|            Managers|    1| 1.1|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|       Professionals|    7| 1.1|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|         Technicians|    0| 1.1|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|       Sales workers|    0| 1.1|     private|         YES|
|23andMe|2016|Hispanic_or_Latino|  male|Administrative su...|    0| 1.1|     private|         YES|
|23andMe|2

In [1]:
df.value = pd.to_numeric(df_nodollar._c6, errors='coerce') 

NameError: ignored

In [ ]:
# df1 = df.select('*',df.area.cast("string"))

new_df = df_nodollar.select('_c6',df_nodollar.area.cast("string")).show()

AttributeError: ignored

In [ ]:
#df_newval = pd.DataFrame(df_nodollar)
#df_newval['_c6'] = df['_c6'].astype(int)

#print (df_newval)
#print (df_newval.dtypes)

TypeError: ignored